# COVID-19 snRNA-seq Analysis Pipeline

## 1. Setup & Configuration

In [ ]:

# 0. Imports & basic config
import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import scvi

print("scanpy:", sc.__version__)
print("scvi-tools:", scvi.__version__)

DATA_DIR = "Path/to/file" # Raw CSV location
SCVI_DIR = "Path/to/file" # Pre-trained SCVI models
OUTPUT_DIR = "./output"  # Results will be saved here

# If you ever want to retrain from CSVs, flip this to True
RUN_SCVI_FROM_SCRATCH = False


## 2. SCVI Integration

In [ ]:
if RUN_SCVI_FROM_SCRATCH:
    os.makedirs(SCVI_DIR, exist_ok=True)

    def run_scvi_on_sample(csv_path):
        sample_id = os.path.basename(csv_path).replace("_raw_counts.csv", "")
        print(f"\n🔄 SCVI training on: {sample_id}")

        # Load counts (cells x genes)
        adata = sc.read_csv(csv_path).T

        # Basic filtering
        sc.pp.filter_genes(adata, min_cells=10)

        # Mark HVGs (for SCVI)
        sc.pp.highly_variable_genes(
            adata,
            n_top_genes=2000,
            subset=True,
            flavor="seurat_v3"
        )

        # Setup SCVI
        scvi.model.SCVI.setup_anndata(adata)
        model = scvi.model.SCVI(adata)
        model.train()

        # Get latent embedding
        latent = model.get_latent_representation()
        print("   latent shape:", latent.shape)

        # Save everything per sample
        out_dir = os.path.join(SCVI_DIR, sample_id)
        os.makedirs(out_dir, exist_ok=True)

        # Save adata (gene space)
        adata.write(os.path.join(out_dir, "adata.h5ad"))

        # Save latent
        np.save(os.path.join(out_dir, "latent.npy"), latent)

        # Save obs names for safety
        with open(os.path.join(out_dir, "obs_names.txt"), "w") as f:
            for idx in adata.obs_names:
                f.write(idx + "\n")

        # Save model
        model.save(out_dir, overwrite=True)

        print(f" Saved SCVI outputs to: {out_dir}")
        return

    # Run SCVI over all CSVs
    for f in sorted(os.listdir(DATA_DIR)):
        if f.endswith(".csv"):
            csv_path = os.path.join(DATA_DIR, f)
            run_scvi_on_sample(csv_path)

    print("\n🎉 Completed SCVI training for all samples.")


In [ ]:
sample_dirs = sorted(
    d for d in os.listdir(SCVI_DIR)
    if os.path.isdir(os.path.join(SCVI_DIR, d))
)

print("Found SCVI sample folders:")
for d in sample_dirs:
    print("  -", d)

adatas = []
latents = []
sample_ids = []

for sample in sample_dirs:
    folder = os.path.join(SCVI_DIR, sample)
    adata_path = os.path.join(folder, "adata.h5ad")
    latent_path = os.path.join(folder, "latent.npy")

    if not os.path.exists(adata_path) or not os.path.exists(latent_path):
        print(f" Skipping {sample} (missing adata.h5ad or latent.npy)")
        continue

    print(f"\n📥 Loading {sample}")
    a = sc.read_h5ad(adata_path)
    z = np.load(latent_path)

    if a.n_obs != z.shape[0]:
        print(f"  Mismatch cells vs latent in {sample}: {a.n_obs} vs {z.shape[0]}")
        continue

    # attach latent to obsm
    a.obsm["X_scVI"] = z

    # Track sample ID
    a.obs["sample"] = sample

    adatas.append(a)
    sample_ids.append(sample)

print("\nTotal loaded samples:", len(adatas))

# Concatenate all samples (gene space)
adata = ad.concat(
    adatas,
    join="inner",          # keep common genes across samples
    label="batch",
    keys=sample_ids,
    index_unique=None      # keep original barcodes (may become unique with suffix)
)

print("\nIntegrated AnnData:")
print(adata)
print("X shape (counts):", adata.X.shape)
print("Latent shape in obsm['X_scVI']:", adata.obsm["X_scVI"].shape)


## 3. Quality Control

In [ ]:

# Mitochondrial genes 
adata.var["mt"] = adata.var_names.str.upper().str.startswith(("MT-", "MT_"))

# Ribosomal protein genes (rp prefixes)
adata.var["ribo"] = adata.var_names.str.upper().str.startswith(("RPL", "RPS"))

# Calculate QC metrics on counts
sc.pp.calculate_qc_metrics(
    adata,
    qc_vars=["mt", "ribo"],
    percent_top=None,
    log1p=False,
    inplace=True
)

print("\nQC metrics added to adata.obs:")
print(adata.obs[["total_counts", "n_genes_by_counts", "pct_counts_mt", "pct_counts_ribo"]].head())

# Filter extreme cells ─ you can tune thresholds
upper_ngenes = np.quantile(adata.obs["n_genes_by_counts"].values, 0.98)
print("2% upper cutoff for n_genes_by_counts:", upper_ngenes)

adata = adata[adata.obs["n_genes_by_counts"] < upper_ngenes, :].copy()
adata = adata[adata.obs["pct_counts_mt"] < 20, :].copy()
adata = adata[adata.obs["pct_counts_ribo"] < 2, :].copy()

print("\nAfter QC filtering:")
print("  n_obs:", adata.n_obs)
print("  n_vars:", adata.n_vars)


In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt", "pct_counts_ribo"],
    jitter=0.4,
    multi_panel=True
)

## 4. Neighbors, UMAP, Clustering

In [ ]:

print("\n📌 Computing neighbors on SCVI latent...")
sc.pp.neighbors(adata, use_rep="X_scVI", n_neighbors=15)

print("📌 Computing UMAP...")
sc.tl.umap(adata)

print("📌 Running Leiden clustering...")
sc.tl.leiden(adata, resolution=0.5)

# Quick visualization
sc.pl.umap(
    adata,
    color=["sample"],
    frameon=False,
    title="UMAP colored by sample"
)

sc.pl.umap(
    adata,
    color=["leiden"],
    frameon=False,
    title="UMAP colored by Leiden clusters"
)


## 5. Gene ID Mapping

In [ ]:
is_ensembl = adata.var_names.str.startswith("ENSG").mean()
print(f"\nFraction of genes starting with 'ENSG': {is_ensembl:.2f}")

if is_ensembl > 0.5:
    print("🔎 var_names look like Ensembl IDs → downloading GENCODE v33 mapping...")

    gtf_url = "https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_33/gencode.v33.annotation.gtf.gz"

    gtf = pd.read_csv(
        gtf_url,
        sep="\t",
        comment="#",
        header=None,
        low_memory=False
    )
    gtf = gtf[gtf[2] == "gene"].copy()
    gtf["gene_id"] = gtf[8].str.extract(r'gene_id "([^"]+)"')
    gtf["gene_name"] = gtf[8].str.extract(r'gene_name "([^"]+)"')

    gene_map = dict(zip(gtf["gene_id"], gtf["gene_name"]))
    print("Mapping loaded:", len(gene_map), "genes")

    # Strip version suffix: ENSG00000141510.12 → ENSG00000141510
    adata.var["ensembl_base"] = (
        adata.var_names.astype(str)
        .str.replace(r"\.\d+$", "", regex=True)
    )
    adata.var["gene_symbol"] = adata.var["ensembl_base"].map(gene_map)

    # Fall back to Ensembl if symbol missing
    missing = adata.var["gene_symbol"].isna().sum()
    print("Missing gene symbols after mapping:", missing)
    adata.var.loc[adata.var["gene_symbol"].isna(), "gene_symbol"] = (
        adata.var.loc[adata.var["gene_symbol"].isna(), "ensembl_base"]
    )
else:
    print("ℹ️ var_names do not look like Ensembl → treating var_names as gene symbols.")
    adata.var["gene_symbol"] = adata.var_names.astype(str)

print("\nExample of gene_symbol column:")
print(adata.var[["gene_symbol"]].head())


## 6. Broad Cell Type Annotation

In [ ]:

marker_dict = {
    "Fibroblast_ECM": [
        "COL1A1","COL1A2","COL3A1","COL5A1","COL5A2","COL6A3","FN1","DCN","LUM"
    ],
    "Endothelial": [
        "VWF","PECAM1","KDR"
    ],
    "Macrophage_like": [
        "MRC1","PPARG","MARCO"
    ]
}

# IMPORTANT FIX #1 — ensure uppercase for comparison
gene_symbols = adata.var_names.str.upper()

print("\n🧬 Scoring marker sets...")

for celltype, genes in marker_dict.items():

    # IMPORTANT FIX #2 — convert markers to uppercase
    genes_upper = [g.upper() for g in genes]

    genes_present = list(set(genes_upper) & set(gene_symbols))

    if len(genes_present) == 0:
        print(f"⚠️ No markers found for {celltype}, skipping.")
        continue

    print(f"  → {celltype}: using {len(genes_present)} markers")

    sc.tl.score_genes(
        adata,
        gene_list=genes_present,
        score_name=f"{celltype}_score",
        use_raw=False
    )

print("\n✅ Marker scoring complete.")

# Assign best cell type
score_cols = [c for c in adata.obs.columns if c.endswith("_score")]

if len(score_cols) > 0:
    score_mat = adata.obs[score_cols].to_numpy()
    best_idx = np.nanargmax(score_mat, axis=1)
    best_labels = np.array([c.replace("_score", "") for c in score_cols])[best_idx]
    adata.obs["broad_cell_type"] = best_labels
else:
    adata.obs["broad_cell_type"] = "Unknown"

# Visualize
sc.pl.umap(
    adata,
    color=["broad_cell_type"],
    frameon=False,
    title="Broad cell-type annotation (OPTION A)"
)


Extract fibroblasts

In [ ]:
import scanpy as sc
import numpy as np

# Select fibroblasts
fib = adata[adata.obs["broad_cell_type"] == "Fibroblast_ECM"].copy()

print("Fibroblast cells:", fib.n_obs)

Normalize → PCA → neighbors → Leiden (Fibroblast only)

In [ ]:
# Normalize + log
sc.pp.normalize_total(fib)
sc.pp.log1p(fib)

# Feature selection
sc.pp.highly_variable_genes(fib, n_top_genes=2000, flavor="seurat_v3")
fib = fib[:, fib.var["highly_variable"]].copy()

# Scale + PCA
sc.pp.scale(fib, max_value=10)
sc.tl.pca(fib, n_comps=30)

# Neighbors + UMAP + Leiden
sc.pp.neighbors(fib, n_neighbors=15, n_pcs=30)
sc.tl.umap(fib)
sc.tl.leiden(fib, resolution=0.6)   # higher resolution for subclustering

Plot Fibroblast UMAP

In [ ]:
sc.pl.umap(fib, color=["leiden"], title="Fibroblast subclusters")

## 7. Fibroblast Subtype Analysis

In [ ]:
# FIBROBLAST SUBTYPE SCORING (FINAL & CLEANED)
# Works with your HVG-limited dataset (161 genes)

import numpy as np
import scanpy as sc

# 1. Define fibroblast subtype markers (Melms et al. + curated)
fibro_markers = {
    "ECM_high": [
        "COL3A1","COL5A2","COL6A3","DCN","FN1"   # only these 5 appear in your HVGs
    ],
    "Alveolar_fibroblast": [
        "FBLN1"  # only marker present in your HVGs
    ],
    # The following subtypes remain but won't score (genes absent)
    "Adventitial_fibroblast": ["CXCL12","PI16","DPP4"],
    "Perivascular_fibroblast": ["RGS5","PDGFRB","ACTA2","TAGLN"],
}

# 2. Work on the fibroblast-only object
 
fib_symbols = fib.var_names  # cleaned gene symbols
print("Total fibroblast genes:", len(fib_symbols))

# 3. Score subtypes

print("\n🧬 Scoring fibroblast subtypes...")

for subtype, genes in fibro_markers.items():

    # Clean markers to uppercase
    genes_up = [g.upper() for g in genes]

    # Intersect with your HVG gene list
    present = sorted(list(set(genes_up) & set(fib_symbols)))

    if len(present) == 0:
        print(f"⚠️ No genes found for {subtype}, skipping.")
        continue

    print(f"  → {subtype}: using {len(present)} genes ({present})")

    # score_genes works since fib.X contains normalized HVGs
    sc.tl.score_genes(
        fib,
        gene_list=present,
        score_name=f"{subtype}_score",
        use_raw=False
    )

print("\n✅ Fibroblast subtype scoring complete.")

# 4. Assign subtype per cell (argmax)
score_cols = [c for c in fib.obs.columns if c.endswith("_score")]
print("Scored subtype columns:", score_cols)

if len(score_cols) > 0:
    score_mat = fib.obs[score_cols].to_numpy()
    best_idx = np.nanargmax(score_mat, axis=1)
    best_labels = np.array(
        [c.replace("_score", "") for c in score_cols]
    )[best_idx]

    fib.obs["fibroblast_subtype"] = best_labels
else:
    fib.obs["fibroblast_subtype"] = "Unknown"

# 5. Plot subtype results
sc.pl.umap(
    fib,
    color=["fibroblast_subtype"],
    frameon=False,
    title="Fibroblast Subtype Annotation"
)


## 8. Condition Assignment & Visualization

In [ ]:

import numpy as np
import scanpy as sc

# 1. Make sure we have:
#    - adata: full integrated AnnData with UMAP + Leiden
#    - adata.obs["broad_cell_type"]: global broad cell types
#    - fib: fibroblast-only AnnData with fib.obs["fibroblast_subtype"]

print("Full object:", adata.shape)
print("Fibroblast object:", fib.shape)
print("Fib obs example:\n", fib.obs[["broad_cell_type", "fibroblast_subtype"]].head())


# 2. Map fibroblast subtypes back to full adata

# Initialize all cells as "Non_fibroblast"
adata.obs["fibroblast_subtype"] = "Non_fibroblast"

# Overwrite for fibroblast cells using fib.obs
common_cells = fib.obs_names.intersection(adata.obs_names)

print(f"\nMapping fibroblast subtypes back for {len(common_cells)} cells.")

adata.obs.loc[common_cells, "fibroblast_subtype"] = fib.obs.loc[common_cells, "fibroblast_subtype"].astype(str)

# Optional: make categorical for nicer legends
adata.obs["broad_cell_type"] = adata.obs["broad_cell_type"].astype("category")
adata.obs["fibroblast_subtype"] = adata.obs["fibroblast_subtype"].astype("category")

print("\nBroad cell types:", adata.obs["broad_cell_type"].cat.categories)
print("Fibroblast subtypes:", adata.obs["fibroblast_subtype"].cat.categories)

# 3. UMAP – global broad cell types

sc.pl.umap(
    adata,
    color="broad_cell_type",
    frameon=False,
    title="Global broad cell-type annotation",
    legend_loc="on data",
)

# 4. UMAP – fibroblast subtypes on full embedding
#    (non-fibroblasts are labeled "Non_fibroblast")

sc.pl.umap(
    adata,
    color="fibroblast_subtype",
    frameon=False,
    title="Fibroblast subtypes on global UMAP",
    legend_loc="right margin",
)

# 5. UMAP – zoom-in: fibroblasts only

fib_plot = adata[adata.obs["fibroblast_subtype"] != "Non_fibroblast", :].copy()

print("\nZoomed fibroblast-only object:", fib_plot.shape)

sc.pl.umap(
    fib_plot,
    color="fibroblast_subtype",
    frameon=False,
    title="Fibroblast subset – subtype annotation",
    legend_loc="right margin",
)

## 9. Differential Expression Analysis

In [ ]:
import numpy as np
import pandas as pd

# Check existing sample labels
print(adata.obs["sample"].value_counts().head(10))

def label_condition(sample):
    s = str(sample).lower()
    if "ctr" in s:
        return "Control"
    elif "cov" in s:
        return "COVID"
    else:
        return "Unknown"

adata.obs["condition"] = adata.obs["sample"].apply(label_condition)

print(adata.obs["condition"].value_counts())


In [ ]:
# UMAP by condition
sc.pl.umap(
    adata,
    color=["condition"],
    frameon=False,
    title="UMAP colored by condition (Control vs COVID)"
)

# UMAP by broad cell type (you already did, but now with condition in mind)
sc.pl.umap(
    adata,
    color=["broad_cell_type"],
    frameon=False,
    title="UMAP colored by broad cell type"
)


In [ ]:
fib = adata[adata.obs["broad_cell_type"] == "Fibroblast_ECM"].copy()

sc.pl.umap(
    fib,
    color=["condition", "fibroblast_subtype"],
    frameon=False,
    title=["Fibroblasts: condition", "Fibroblasts: subtype"]
)


In [ ]:
# Broad cell types by condition
ct_table = pd.crosstab(adata.obs["broad_cell_type"], adata.obs["condition"])
print(ct_table)

# Fibroblast subtype by condition (only fibro cells)
fib_table = pd.crosstab(
    fib.obs.get("fibroblast_subtype", "Unknown"),
    fib.obs["condition"]
)
print(fib_table)

9A — UMAP Colored by Broad Cell Types

In [ ]:
sc.pl.umap(
    adata,
    color="broad_cell_type",
    legend_loc="on data",
    title="Broad Cell Types",
    frameon=False,
    size=10,
    palette="tab20"
)

9B — UMAP Colored by Condition (Control vs COVID)

In [ ]:
sc.pl.umap(
    adata,
    color="condition",
    frameon=False,
    title="Condition (Control vs COVID)",
    size=10,
    palette={"Control": "#1f77b4", "COVID": "#d62728"}
)


9C — UMAP of Fibroblasts Only (Subtype)

In [ ]:
sc.pl.umap(
    fib,
    color="fibroblast_subtype",
    title="Fibroblast Subtypes",
    frameon=False,
    size=10,
    palette="tab10"
)

9D — Barplot: Cell Type Proportions per Condition

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

ct_df = (adata.obs
         .groupby(["condition", "broad_cell_type"])
         .size()
         .reset_index(name="count"))

plt.figure(figsize=(8,5))
sns.barplot(
    data=ct_df,
    x="broad_cell_type",
    y="count",
    hue="condition",
    palette={"Control": "#1f77b4", "COVID": "#d62728"}
)
plt.xticks(rotation=45, ha='right')
plt.title("Cell Type Proportions per Condition")
plt.tight_layout()
plt.show()


9E — Dotplot of Marker Genes Across Cell Types

In [ ]:
# 9E — FIXED DOTPLOT

all_markers = {
    "Fibroblast_ECM": ["COL1A1","COL3A1","COL5A1","COL5A2","DCN","FN1"],
    "Endothelial": ["PECAM1","VWF","KDR"],
    "Macrophage_like": ["MRC1","MARCO","PPARG"]
}

# Filter to keep only genes present
filtered_markers = {
    ct: [g for g in genes if g in adata.var_names]
    for ct, genes in all_markers.items()
}

# Remove empty marker sets
filtered_markers = {k:v for k,v in filtered_markers.items() if len(v)>0}

print("Dotplot genes actually used:", filtered_markers)

sc.pl.dotplot(
    adata,
    var_names=filtered_markers,
    groupby="broad_cell_type",
    standard_scale="var",
    title="Marker Expression per Cell Type (Filtered)"
)


In [ ]:
# 9F — FIXED HEATMAP

fib_score_cols = [c for c in fib.obs.columns if c.endswith("_score")]

print("Fibroblast score columns:", fib_score_cols)

if len(fib_score_cols) == 0:
    print("❌ No fibroblast scores to plot.")
else:
    sc.pl.heatmap(
        fib,
        var_names=fib_score_cols,
        groupby="fibroblast_subtype",
        swap_axes=True,
        cmap="viridis",
        figsize=(6,5),
        dendrogram=False
    )



In [ ]:
# 9G — FIXED UMAP SPLITS

adata_ctrl = adata[adata.obs["condition"] == "Control"].copy()
adata_covid = adata[adata.obs["condition"] == "COVID"].copy()

sc.pl.umap(
    adata_ctrl,
    color="broad_cell_type",
    title="Control Only",
    frameon=False
)

sc.pl.umap(
    adata_covid,
    color="broad_cell_type",
    title="COVID Only",
    frameon=False
)


## 10. Functional Enrichment

10A — Create pseudo-bulk matrix (sum counts per sample)

In [ ]:
import os
import scanpy as sc
import numpy as np

raw_dir = "/scratch/roy.suc/final_project/data"

print("📁 Loading raw CSV files…")
raw_list = []

for f in sorted(os.listdir(raw_dir)):
    if f.endswith(".csv"):
        csvpath = os.path.join(raw_dir, f)
        print("Reading:", csvpath)

        raw = sc.read_csv(csvpath).T
        sample = f.replace("_raw_counts.csv", "")
        raw.obs["sample"] = sample
        raw_list.append(raw)

print("\nLoaded raw matrices:", len(raw_list))

# 1. Concatenate all raw matrices

print("\n🔗 Concatenating...")
adata_raw = raw_list[0].concatenate(
    raw_list[1:], 
    batch_key="raw_batch", 
    index_unique=None
)

# Make obs names unique
adata_raw.obs_names_make_unique()

# Clean suffix like "-1", "-2"
adata_raw.obs_names = adata_raw.obs_names.str.replace(r"-\d+$", "", regex=True)

print("\n Cleaned raw names (example):", adata_raw.obs_names[:5].tolist())

# 2. Align obs order (cells) with integrated AnnData

int_ids = adata.obs_names

matches = np.isin(adata_raw.obs_names, int_ids).sum()
print(f"\nMatching: {matches} / {len(int_ids)}")

if matches != len(int_ids):
    raise ValueError("❌ Raw and integrated obs_names do not match. Stop.")

# Reorder raw to match integrated
adata_raw = adata_raw[int_ids, :]

print("Aligned raw shape:", adata_raw.shape)

# 3. OPTION A FIX — Subset genes to match integrated gene list (161 genes)

print("\n✨ Subsetting raw to integrated gene list...")
adata_raw_subset = adata_raw[:, adata.var_names].copy()

print("Subset raw shape:", adata_raw_subset.shape)
print("Integrated adata shape:", adata.shape)

# 4. Attach as raw layer

adata.layers["raw"] = adata_raw_subset.X.copy()

print("\n🎉 SUCCESS → raw counts attached to adata.layers['raw']")
print("Final layer shape:", adata.layers["raw"].shape)


In [ ]:
# Convert sample to condition
adata.obs["condition"] = adata.obs["sample"].apply(
    lambda x: "covid" if "cov" in x.lower() else "control"
)

print(adata.obs["condition"].value_counts())


In [ ]:
cell_types = adata.obs["broad_cell_type"].unique()
cell_types


In [ ]:
adata.layers.keys()


## 11. Final Figure Export

In [ ]:
# FINAL DIFFERENTIAL EXPRESSION SCRIPT (COVID vs CONTROL)
import scanpy as sc
import pandas as pd
import numpy as np
import os

print("🔍 Starting DE analysis using raw counts layer...")

# 0. Verify raw layer exists
if "raw" not in adata.layers:
    raise ValueError("❌ ERROR: adata.layers['raw'] is missing. Attach raw counts first.")

print("✅ Raw layer detected.")

# 1. Create raw_subset layer (subset-friendly)
adata.layers["raw_subset"] = adata.layers["raw"].copy()
print("✅ Created adata.layers['raw_subset']")


# 2. Prepare output directory
outdir = "DE_results"
os.makedirs(outdir, exist_ok=True)
print(f"📁 Output folder: {outdir}")


# 3. Identify all major cell types for DE testing
celltypes = adata.obs["broad_cell_type"].unique().tolist()
print("🔬 Broad cell types to analyze:", celltypes)


# 4. Loop over each cell type & run DE
de_results = {}

for ct in celltypes:
    print(f"\n===============================================")
    print(f"🔎 DE for cell type: {ct}")
    print(f"===============================================")

    # ---- slice AFTER raw_subset layer is present ----
    subset = adata[adata.obs["broad_cell_type"] == ct, :]

    if subset.n_obs < 50:
        print(f"⚠️ Too few cells ({subset.n_obs}) → skipping.")
        continue
    
    print(f"Cells: {subset.n_obs}, Genes: {subset.n_vars}")

    # Must contain both conditions
    conds = subset.obs["condition"].unique().tolist()
    if not {"covid", "control"}.issubset(conds):
        print("⚠️ Missing covid/control groups → skipping.")
        continue

    print("Running Wilcoxon test...")

    # ------------- RUN DE -----------------
    sc.tl.rank_genes_groups(
        subset,
        groupby="condition",
        groups=["covid"],
        reference="control",
        method="wilcoxon",
        use_raw=False,
        layer="raw_subset"
    )

    # Save result
    de_results[ct] = subset

    # Convert to table
    res = sc.get.rank_genes_groups_df(subset, group="covid")

    # Save to CSV
    outfile = f"{outdir}/DE_{ct.replace(' ', '_')}_covid_vs_control.csv"
    res.to_csv(outfile, index=False)

    print(f"📄 Saved DE table → {outfile}")


print("\n🎉 DE ANALYSIS COMPLETE!")
print(f"All DE results stored in: {outdir}/")


In [ ]:
# STEP 11 — Volcano Plots for DE Results (COVID vs Control)
#   → This version DISPLAYS each plot + SAVES it to file

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# 1. Paths
de_dir = "DE_results"
volcano_dir = f"{de_dir}/volcano"
os.makedirs(volcano_dir, exist_ok=True)

print(f"📁 Volcano plots will be saved to: {volcano_dir}")

# 2. Find DE CSV files
de_files = [f for f in os.listdir(de_dir) if f.endswith(".csv")]
print("Found DE files:", de_files)


# 3. Volcano plotting function
def plot_volcano(df, celltype, savepath):
    plt.figure(figsize=(7, 6))

    # Extract
    logfc = df["logfoldchanges"]
    pvals_adj = df["pvals_adj"].replace(0, 1e-300)

    x = logfc
    y = -np.log10(pvals_adj)

    # Significance
    sig = (pvals_adj < 0.05) & (abs(logfc) > 0.5)

    # Base points
    plt.scatter(x, y, s=10, c="gray", alpha=0.6)
    # Significant points
    plt.scatter(x[sig], y[sig], s=14, c="red", alpha=0.8)

    # Threshold lines
    plt.axvline(0.5, linestyle="--", color="black", alpha=0.4)
    plt.axvline(-0.5, linestyle="--", color="black", alpha=0.4)
    plt.axhline(-np.log10(0.05), linestyle="--", color="black", alpha=0.4)

    plt.xlabel("log2 Fold Change (COVID vs Control)", fontsize=12)
    plt.ylabel("-log10(adj p-value)", fontsize=12)
    plt.title(f"Volcano — {celltype}", fontsize=14)

    plt.tight_layout()

    # ---- NEW: Show figure ----
    plt.show()

    # ---- Save file ----
    plt.savefig(savepath, dpi=300)
    plt.close()

# 4. Loop over all DE results
for fname in de_files:
    fpath = os.path.join(de_dir, fname)

    df = pd.read_csv(fpath)

    celltype = fname.replace("DE_", "").replace("_covid_vs_control.csv", "")
    celltype_label = celltype.replace("_", " ")

    savepath = os.path.join(volcano_dir, f"volcano_{celltype}.png")

    print(f"📊 Generating volcano: {celltype_label}")

    plot_volcano(df, celltype_label, savepath)

    print(f"   ✔ Saved: {savepath}")

print("\n🎉 All volcano plots displayed and saved!")


In [ ]:
# STEP 13 — Summarize DE results (Top genes + DE counts)

import os
import pandas as pd
import numpy as np

de_dir = "DE_results"
summary_dir = f"{de_dir}/Summary"
os.makedirs(summary_dir, exist_ok=True)

print(f"📁 Saving summaries to: {summary_dir}")

# Helper functions
def count_sig_genes(df, p_adj_thresh=0.05, logfc_thresh=0.25):
    """Counts significant DE genes."""
    df = df.dropna(subset=["pvals_adj", "logfoldchanges"])
    sig = df[
        (df["pvals_adj"] < p_adj_thresh) &
        (df["logfoldchanges"].abs() > logfc_thresh)
    ]
    return len(sig)


def get_top_genes(df, n=20):
    """Returns top up and top down DE gene lists."""
    df = df.dropna(subset=["logfoldchanges"])
    up = df.sort_values("logfoldchanges", ascending=False).head(n)["names"].tolist()
    down = df.sort_values("logfoldchanges", ascending=True).head(n)["names"].tolist()
    return up, down


# MASTER SUMMARY TABLE
summary_rows = []

de_files = [f for f in os.listdir(de_dir) if f.endswith(".csv")]

print("\n🔍 Found DE files:")
for f in de_files:
    print("  -", f)

for fname in de_files:
    path = os.path.join(de_dir, fname)
    df = pd.read_csv(path)

    celltype = fname.replace("DE_", "").replace("_covid_vs_control.csv", "")
    cell_label = celltype.replace("_", " ")

    n_sig = count_sig_genes(df)
    up, down = get_top_genes(df, n=20)

    summary_rows.append({
        "cell_type": cell_label,
        "n_significant_genes": n_sig,
        "top20_up": ", ".join(up),
        "top20_down": ", ".join(down),
    })

    # Save per-cell-type top gene table
    df_top = pd.DataFrame({
        "top20_up": up,
        "top20_down": down
    })
    df_top.to_csv(
        os.path.join(summary_dir, f"{celltype}_top_genes.csv"),
        index=False
    )

    print(f"✔ Processed: {cell_label} — {n_sig} significant genes")

# Save master summary
summary_table = pd.DataFrame(summary_rows)
summary_table.to_csv(os.path.join(summary_dir, "DE_summary.csv"), index=False)

print("\n🎉 DONE — Summary table saved to:")
print(f"   {summary_dir}/DE_summary.csv")


## 10. Functional Enrichment

In [ ]:
# STEP 14 — Functional "Pathway" Summary from DE genes
#   - No gseapy, no online requests
#   - Uses simple curated gene sets to tag biology per cell type

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Paths
de_dir = "DE_results"
func_dir = os.path.join(de_dir, "Functional_summary")
plot_dir = os.path.join(func_dir, "plots")

os.makedirs(func_dir, exist_ok=True)
os.makedirs(plot_dir, exist_ok=True)

print(f"📁 Functional summaries will be saved in: {func_dir}")

# 1. Define simple manual gene sets (upper-case symbols)
#    These are coarse functional categories, not full GSEA.

functional_gene_sets = {
    "ECM_remodeling": [
        "COL1A1","COL1A2","COL3A1","COL5A1","COL5A2","COL6A3",
        "FN1","DCN","LUM","MMP2","MMP9","TNC"
    ],
    "Fibrosis_TGFbeta": [
        "TGFB1","TGFB2","TGFB3","TGFBR1","TGFBR2","SERPINE1",
        "CTGF","SMAD3","SMAD7"
    ],
    "Interferon_response": [
        "ISG15","IFI6","IFIT1","IFIT3","MX1","MX2",
        "OAS1","OAS2","OAS3","IFITM1","IFITM3"
    ],
    "Cytokine_chemokine_inflammation": [
        "CXCL10","CXCL9","CXCL8","CCL2","CCL3","CCL4",
        "IL1B","IL6","TNF","CXCL1","CXCL2"
    ],
    "Angiogenesis_vascular": [
        "VEGFA","KDR","FLT1","VWF","PECAM1","ANGPT2","PDGFB"
    ],
    "Proliferation_cell_cycle": [
        "MKI67","TOP2A","PCNA","CCNB1","CCNB2","CDK1","BIRC5"
    ],
    "Apoptosis_stress": [
        "BAX","BCL2","CASP3","CASP8","CASP9","TP53","GADD45A"
    ]
}

# Make sure everything is uppercase
functional_gene_sets = {
    k: [g.upper() for g in v] for k, v in functional_gene_sets.items()
}

# 2. Helper: filter DE and compute overlap with gene sets
def summarize_de_for_celltype(df, celltype, p_cut=0.05, logfc_cut=0.25):
    """
    df: DE table with columns 'names', 'pvals_adj', 'logfoldchanges'
    Returns: summary DataFrame with functional category overlaps.
    """

    # standardize column names (just in case)
    cols = df.columns.str.lower()
    col_map = {old: new for old, new in zip(df.columns, cols)}
    df = df.rename(columns=col_map)

    # expect 'names', 'pvals_adj', 'logfoldchanges'
    if not {"names", "pvals_adj", "logfoldchanges"}.issubset(df.columns):
        print(f"⚠️ {celltype}: DE table does not have required columns. Skipping.")
        return None

    # Filter to significant DE
    sig = df[
        (df["pvals_adj"] < p_cut) &
        (df["logfoldchanges"].abs() > logfc_cut)
    ].copy()

    if sig.empty:
        print(f"⚠️ {celltype}: No significant DE genes at this threshold.")
        return None

    sig_genes = sig["names"].str.upper().tolist()
    sig_set = set(sig_genes)

    summary_rows = []
    for cat, gene_list in functional_gene_sets.items():
        overlap = sig_set.intersection(gene_list)
        n_overlap = len(overlap)
        if n_overlap == 0:
            continue

        summary_rows.append({
            "cell_type": celltype,
            "category": cat,
            "n_overlap": n_overlap,
            "n_sig_genes": len(sig_genes),
            "frac_sig_in_category": n_overlap / len(sig_genes),
            "overlap_genes": ";".join(sorted(list(overlap)))
        })

    if len(summary_rows) == 0:
        print(f"ℹ️ {celltype}: No overlaps with manual functional gene sets.")
        return None

    summary_df = pd.DataFrame(summary_rows)
    return summary_df

# 3. Loop over DE CSVs and build functional summary

de_files = [f for f in os.listdir(de_dir) if f.endswith(".csv")]

print("\n🔥 Running functional summary for DE files:")
for f in de_files:
    print("  -", f)

all_summaries = []

for fname in de_files:
    path_csv = os.path.join(de_dir, fname)
    celltype = fname.replace("DE_", "").replace("_covid_vs_control.csv", "")
    celltype_label = celltype.replace("_", " ")

    print(f"\n🧪 Processing {celltype_label}...")
    df_de = pd.read_csv(path_csv)

    summary_df = summarize_de_for_celltype(df_de, celltype_label)
    if summary_df is None:
        continue

    # Save per-cell-type summary
    out_path = os.path.join(func_dir, f"{celltype}_functional_summary.csv")
    summary_df.to_csv(out_path, index=False)
    print(f"✔ Saved functional summary: {out_path}")

    all_summaries.append(summary_df)

# 4. Save combined functional summary table

if len(all_summaries) > 0:
    combined = pd.concat(all_summaries, ignore_index=True)
    combined_path = os.path.join(func_dir, "ALL_celltypes_functional_summary.csv")
    combined.to_csv(combined_path, index=False)
    print(f"\n📊 Combined functional summary saved: {combined_path}")
else:
    combined = None
    print("\n⚠️ No functional overlaps found for any cell type.")

# 5. Plot barplots: categories per cell type

if combined is not None and not combined.empty:
    # Simple barplot: n_overlap by category per cell type
    plt.figure(figsize=(10, 6))
    sns.barplot(
        data=combined,
        x="category",
        y="n_overlap",
        hue="cell_type"
    )
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Number of overlapping DE genes")
    plt.title("Functional category overlap per cell type")
    plt.tight_layout()

    plot_path = os.path.join(plot_dir, "functional_overlap_barplot.png")
    plt.savefig(plot_path, dpi=300)
    plt.show()
    plt.close()
    print(f"📈 Saved barplot: {plot_path}")

    # Optional: heatmap-style pivot
    pivot = combined.pivot_table(
        index="cell_type",
        columns="category",
        values="n_overlap",
        fill_value=0
    )

    plt.figure(figsize=(8, 5))
    sns.heatmap(pivot, annot=True, fmt=".1f", cmap="Blues")
    plt.title("DE overlap with functional gene sets")
    plt.tight_layout()

    heatmap_path = os.path.join(plot_dir, "functional_overlap_heatmap.png")
    plt.savefig(heatmap_path, dpi=300)
    plt.show()
    plt.close()
    print(f"📈 Saved heatmap: {heatmap_path}")

print("\n🎉 STEP 14 COMPLETE — functional summaries + plots are ready.")


In [ ]:
import scanpy as sc
import os

# Tell Scanpy where to save figures
sc.settings.figdir = "./Final_Figures"

# create folder
os.makedirs("Final_Figures", exist_ok=True)

print("Figures will now be saved in: ", sc.settings.figdir)


In [ ]:
sc.pl.umap(
    adata,
    color="sample",
    frameon=False,
    title="UMAP by Sample",
    save="_umap_sample.png"
)

sc.pl.umap(
    adata,
    color="leiden",
    frameon=False,
    title="UMAP — Leiden Clusters",
    save="_umap_leiden.png"
)

sc.pl.umap(
    adata,
    color="broad_cell_type",
    frameon=False,
    title="UMAP — Broad Cell Types",
    save="_umap_broad_types.png"
)

sc.pl.umap(
    fib,
    color="fibroblast_subtype",
    frameon=False,
    title="Fibroblast Subtype Annotation",
    save="_fibroblast_subtypes.png"
)


In [ ]:
# STEP 15B — Final Combined Panel Figure
sc.settings.figdir = "./Final_Figures"
os.makedirs("Final_Figures", exist_ok=True)

# STEP 15B — Combined multi-panel figure (4 UMAPs)

import matplotlib.pyplot as plt
import scanpy as sc
import numpy as np
import os

print("Saving combined figure to ./Final_Figures")

# Extract UMAP embeddings directly from adata / fib
umap_main = adata.obsm["X_umap"]
umap_fib  = fib.obsm["X_umap"]

# 1. Setup figure
plt.figure(figsize=(16, 16))

# 2A. UMAP by Sample
ax1 = plt.subplot(2, 2, 1)
sc.pl.umap(
    adata,
    color="sample",
    frameon=False,
    show=False,
    ax=ax1
)
ax1.set_title("UMAP — by Sample", fontsize=12)

# 2B. UMAP by Leiden Clusters
ax2 = plt.subplot(2, 2, 2)
sc.pl.umap(
    adata,
    color="leiden",
    frameon=False,
    show=False,
    ax=ax2
)
ax2.set_title("UMAP — Leiden Clusters", fontsize=12)

# 2C. Broad Cell Types
ax3 = plt.subplot(2, 2, 3)
sc.pl.umap(
    adata,
    color="broad_cell_type",
    frameon=False,
    show=False,
    ax=ax3
)
ax3.set_title("UMAP — Broad Cell Types", fontsize=12)

# 2D. Fibroblast Subtypes (fib object)
ax4 = plt.subplot(2, 2, 4)
sc.pl.umap(
    fib,
    color="fibroblast_subtype",
    frameon=False,
    show=False,
    ax=ax4
)
ax4.set_title("UMAP — Fibroblast Subtypes", fontsize=12)

# 3. Layout + Save
plt.tight_layout()

save_path = "Final_Figures/UMAP_combined_panel.png"
plt.savefig(save_path, dpi=300, bbox_inches="tight")
plt.show()

print(f"🎉 Combined multi-panel UMAP saved at:\n{save_path}")



In [ ]:
# STEP 16 — Dotplots for Global Cell Types + Fibro Subtypes (SAFE VERSION)

import os
import scanpy as sc
import matplotlib.pyplot as plt

dotplot_dir = "Final_Figures/dotplots"
os.makedirs(dotplot_dir, exist_ok=True)
print("📁 Saving dotplots to:", dotplot_dir)


# 16A — GLOBAL CELL-TYPE DOTPLOT (safe filtering)

marker_dict_global = {
    "Fibroblast_ECM": ["COL3A1", "COL5A2", "COL6A3", "FN1", "DCN"],
    "Endothelial": ["VWF"],
    "Macrophage_like": ["MRC1"],
}

# Filter for genes that exist
marker_dict_global_filtered = {}
for ct, genes in marker_dict_global.items():
    present = [g for g in genes if g in adata.var_names]
    if len(present) > 0:
        marker_dict_global_filtered[ct] = present
    else:
        print(f"⚠️ Skipping {ct} — no valid genes found.")

print("\n🎨 Plotting global cell-type dotplot...")

fig1 = sc.pl.dotplot(
    adata,
    marker_dict_global_filtered,
    groupby="broad_cell_type",
    standard_scale="var",
    return_fig=True
)

fig1.savefig(f"{dotplot_dir}/dotplot_global_celltypes.png", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

print("✔ Saved: dotplot_global_celltypes.png")


# 16B — FIBROBLAST SUBTYPE DOTPLOT (safe filtering)

marker_dict_fib = {
    "ECM_high": ["COL3A1", "COL5A2", "COL6A3", "DCN", "FN1"],
    "Alveolar_fibroblast": ["FBLN1"],
}

marker_dict_fib_filtered = {}
for ct, genes in marker_dict_fib.items():
    present = [g for g in genes if g in fib.var_names]
    if len(present) > 0:
        marker_dict_fib_filtered[ct] = present
    else:
        print(f"⚠️ Skipping {ct} — no valid genes in fibroblast subset.")

print("\n🎨 Plotting fibroblast subtype dotplot...")

fig2 = sc.pl.dotplot(
    fib,
    marker_dict_fib_filtered,
    groupby="fibroblast_subtype",
    standard_scale="var",
    return_fig=True
)

fig2.savefig(f"{dotplot_dir}/dotplot_fibro_subtypes.png", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

print("✔ Saved: dotplot_fibro_subtypes.png")

print("\n🎉 STEP 16 COMPLETE — safe dotplots generated successfully!")


In [ ]:
# STEP 17 — Save final AnnData + metadata

import os
import pandas as pd
import numpy as np
import scanpy as sc

save_dir = "Final_Export"
os.makedirs(save_dir, exist_ok=True)

print(f"📁 Saving output files to: {save_dir}")


# 1. Save the FINAL integrated AnnData object
final_h5ad_path = f"{save_dir}/adata_final_melms_like.h5ad"
adata.write(final_h5ad_path)
print(f"✔ Saved final integrated object: {final_h5ad_path}")


# 2. Save fibroblast-only AnnData
fib_h5ad_path = f"{save_dir}/adata_fibroblast_only.h5ad"
fib.write(fib_h5ad_path)
print(f"✔ Saved fibroblast subset: {fib_h5ad_path}")


# 3. Export metadata table (obs)
meta_path = f"{save_dir}/cell_metadata.csv"

adata.obs.to_csv(meta_path)
print(f"✔ Saved metadata table: {meta_path}")


# 4. Export cluster assignments + cell-type labels
cluster_path = f"{save_dir}/clusters_celltypes.csv"

adata.obs[["sample", "condition", "leiden", "broad_cell_type"]].to_csv(cluster_path)
print(f"✔ Saved cluster/cell-type table: {cluster_path}")


# 5. Export UMAP coordinates
if "X_umap" in adata.obsm:
    umap_df = pd.DataFrame(
        adata.obsm["X_umap"],
        index=adata.obs_names,
        columns=["UMAP_1", "UMAP_2"]
    )
    umap_df.to_csv(f"{save_dir}/umap_coordinates.csv")
    print("✔ Saved UMAP coordinates")
else:
    print("⚠️ WARNING: UMAP not found in adata.obsm")


# 6. Export SCVI latent embedding (10D)
if "X_scVI" in adata.obsm:
    latent_df = pd.DataFrame(
        adata.obsm["X_scVI"],
        index=adata.obs_names,
        columns=[f"SCVI_{i+1}" for i in range(adata.obsm['X_scVI'].shape[1])]
    )
    latent_df.to_csv(f"{save_dir}/scvi_latent_embedding.csv")
    print("✔ Saved SCVI latent embedding")
else:
    print("⚠️ WARNING: SCVI latent embedding not found in adata.obsm")


# 7. Export DE results merged summary (optional)

summary_file = "de_summary.csv"
summary_path = os.path.join("DE_results", summary_file)

if os.path.exists(summary_path):
    out_summary = f"{save_dir}/de_summary_combined.csv"
    os.system(f"cp {summary_path} {out_summary}")
    print(f"✔ Copied DE summary table: {out_summary}")
else:
    print("⚠️ No combined DE summary found — skipping.")


# 8. Export gene list for each cell type (optional)
gene_dir = f"{save_dir}/Top_Genes"
os.makedirs(gene_dir, exist_ok=True)

for celltype in adata.obs["broad_cell_type"].unique():
    try:
        # Only save top DE if file exists
        fname = f"DE_{celltype}_covid_vs_control.csv"
        fpath = os.path.join("DE_results", fname)
        if os.path.exists(fpath):
            df = pd.read_csv(fpath)
            df.to_csv(f"{gene_dir}/{celltype}_DE_genes.csv", index=False)
            print(f"✔ Saved top genes for {celltype}")
    except:
        pass


print("\n🎉 STEP 17 COMPLETE — All exports finished successfully!")
